**Настройка макета записаной книжки -> Output: Word Wrap -> Вкл.**

### Что изменено?
- ХЗ, пока вроде ничего, просто код почище сделал

In [57]:
# РОТ ШАТАЛ ЭТИХ ЗАВИСИМОСТЕЙ ПИТОНА

# !sudo apt remove python* -y
# !sudo apt install python3 pip -y
# !/bin/python3 -m pip install ipykernel -U --user --force-reinstall
# !pip freeze | cut -d "@" -f1 | xargs pip uninstall -y

# !pip uninstall datasets spacy scikit-learn -y
# !pip uninstall thinc numpy scipy -y
# !pip install "thinc>8.3" "numpy<2.0" scipy
# !pip install scikit-learn spacy datasets

In [ ]:
!python3 -m spacy download ru_core_news_sm

In [59]:
import numpy as np
from typing import List, Tuple

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import spacy
from datasets import load_dataset
import multiprocessing

### 1. Загрузим и разделим текстовый корпус

In [60]:
def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    X_train = trainset['text']
    y_train = trainset['category']

    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']

    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']


    categories = set(y_train)

    unknown_categories = set(y_val) - set(y_train)
    if len(unknown_categories) > 0:
        raise RuntimeError(
            f'Категории {unknown_categories} есть в валидационном наборе, но не в тренировочном'
        )

    unknown_categories = set(y_test) - set(y_train)
    if len(unknown_categories) > 0:
        raise RuntimeError(
            f'Категории {unknown_categories} есть в тестовом наборе, но не в тренировочном'
        )


    categories = sorted(list(categories))

    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [61]:
train_data, val_data, test_data, classes_list = load_sib200_ru()

In [62]:
print('Категории текстового корпуса:', classes_list)

Категории текстового корпуса: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [63]:
print(len(train_data[0]), len(train_data[1]))
print(len(val_data[0]), len(val_data[1]))
print(len(test_data[0]), len(test_data[1]))

701 701
99 99
204 204


### 2. Применим SpaCy для предобработки текста

In [79]:
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    doc = nlp_pipeline(s)
    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct, doc)]
    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [80]:
NLP_PIPELINE = spacy.load('ru_core_news_sm')

In [84]:
print(
    train_data[0][0],
    '\n' + normalize_text(train_data[0][0], NLP_PIPELINE),
)
print(
    '\n' + val_data[0][0],
    '\n' + normalize_text(val_data[0][0], NLP_PIPELINE),
)
print(
    '\n' + test_data[0][0],
    '\n' + normalize_text(test_data[0][0], NLP_PIPELINE),
)

Турция с трёх сторон окружена морями: на западе — Эгейским, на севере — Чёрным и на юге — Средиземным. 
турция с <NUM> сторона окружить морями на запад эгейским на север чёрный и на юг средиземный

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость. 
если увеличить расстояние для бег с <NUM> до <NUM> миля скорость становиться не так важный тогда как выносливость превращаться в абсолютный необходимость

Мутация вносит новую генетическую вариацию, в то время как отбор убирает её из набора проявляющихся вариаций. 
мутация вносить новый генетический вариация в тот время как отбор убирать её из набор проявляться вариация


In [67]:
class_probability = 1.0 / len(classes_list)
max_df = 1.0 - 0.2 * class_probability

print(f'Максимальная документная частота (TF-IDF) слова: {max_df}.')

Максимальная документная частота (TF-IDF) слова: 0.9714285714285714.


### 3. Применим какой-нибудь классификатор

In [68]:
classifier = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1)),
    ('cls', LogisticRegression(solver='saga', max_iter=100, random_state=42))
])

In [69]:
cv = GridSearchCV(
    estimator=classifier,
    param_grid={
        'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'cls__C': [1e-1, 1, 10, 100, 1000],
        'cls__penalty': ['l1', 'l2']
    },
    scoring='f1_macro',
    cv=5,
    refit=True,
    n_jobs=multiprocessing.cpu_count()-1,
    verbose=True
)

In [ ]:
cv.fit(
    [normalize_text(it, NLP_PIPELINE) for it in train_data[0]],
    train_data[1]
)

In [71]:
print('Лучшие параметры:', cv.best_params_)
print('Лучший показатель F1:', cv.best_score_)
print('Размер вокабуляра:', len(cv.best_estimator_.named_steps["vectorizer"].vocabulary_))

Лучшие параметры: {'cls__C': 1000, 'cls__penalty': 'l2', 'vectorizer__ngram_range': (1, 1)}
Лучший показатель F1: 0.6451646278967693
Размер вокабуляра: 4359


In [72]:
y_pred = cv.predict([normalize_text(it, NLP_PIPELINE) for it in val_data[0]])

print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.80      0.44      0.57         9
         geography       0.67      0.75      0.71         8
            health       1.00      0.45      0.62        11
          politics       0.77      0.71      0.74        14
science/technology       0.62      0.80      0.70        25
            sports       0.75      0.75      0.75        12
            travel       0.57      0.65      0.60        20

          accuracy                           0.68        99
         macro avg       0.74      0.65      0.67        99
      weighted avg       0.71      0.68      0.67        99



In [73]:
y_pred = cv.predict([normalize_text(it, NLP_PIPELINE) for it in test_data[0]])

print(classification_report(y_true=test_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.89      0.42      0.57        19
         geography       0.64      0.53      0.58        17
            health       0.47      0.41      0.44        22
          politics       0.78      0.83      0.81        30
science/technology       0.65      0.78      0.71        51
            sports       0.87      0.80      0.83        25
            travel       0.62      0.70      0.66        40

          accuracy                           0.68       204
         macro avg       0.70      0.64      0.66       204
      weighted avg       0.69      0.68      0.68       204

